# Training an ML Model on Tensorflow Datasets
## Prerequisites

In [1]:
import datetime
import gc
import glob
import json
import os
import random
import shutil
import time
from typing import Iterable, Callable, Dict, Any, Tuple, Optional, List, Union

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.python import keras as K

from mmproteo.utils import log, paths, utils, visualization
from mmproteo.utils.ml import callbacks, evaluation, layers, losses
from mmproteo.utils.formats.mz import FilteringProcessor, MzmlidFileStatsCreator
from mmproteo.utils.formats.tf_dataset import Parquet2DatasetFileProcessor, DatasetLoader
from mmproteo.utils.processing import ItemProcessor

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

In [3]:
logger = log.DummyLogger(verbose=False)

INFO: Printing to Stdout


## Configuration

In [4]:
pwd

'/tf/workspace/notebooks'

In [5]:
PROJECT = "PXD010000"
DUMP_PATH = os.path.join("..", "dumps", PROJECT)
TRAINING_COLUMNS_DUMP_PATH = os.path.join(DUMP_PATH, "training_columns")
FILES_PATH = os.path.join(TRAINING_COLUMNS_DUMP_PATH, "*_mzmlid.parquet")
STATISTICS_FILE_PATH = os.path.join(TRAINING_COLUMNS_DUMP_PATH, "statistics.parquet")
DATASET_DUMP_PATH = os.path.join(TRAINING_COLUMNS_DUMP_PATH, "tf_datasets")
PROCESSING_FILE_PATH = os.path.join(DATASET_DUMP_PATH, "processing_info.json")

In [6]:
SEQ = 'peptide_sequence'
MZ = 'mz_array'
INT = 'intensity_array'

In [7]:
TRAINING_DATA_COLUMNS = [MZ, INT]
TARGET_DATA_COLUMNS = [SEQ]
SPLIT_VALUE_COLUMNS = ['species', 'istrain']

In [8]:
with open(PROCESSING_FILE_PATH, 'r') as file:
    PROCESSING_INFO = json.loads(file.read())
PROCESSING_INFO

{'padding_characters': {'peptide_sequence': '_',
  'mz_array': 0.0,
  'intensity_array': 0.0},
 'padding_lengths': {'mz_array': 2354,
  'intensity_array': 2354,
  'peptide_sequence': 50},
 'idx_to_char': {'0': 'A',
  '1': 'C',
  '2': 'D',
  '3': 'E',
  '4': 'F',
  '5': 'G',
  '6': 'H',
  '7': 'I',
  '8': 'K',
  '9': 'L',
  '10': 'M',
  '11': 'M(Oxidation)',
  '12': 'N',
  '13': 'P',
  '14': 'Q',
  '15': 'R',
  '16': 'S',
  '17': 'T',
  '18': 'V',
  '19': 'W',
  '20': 'Y',
  '21': '_'},
 'normalization': {'intensity_array': '<function base_peak_normalize at 0x7fa6046d5158>'},
 'split_value_columns': ['species', 'istrain'],
 'training_data_columns': ['mz_array', 'intensity_array'],
 'target_data_columns': ['peptide_sequence']}

In [9]:
idx_to_char = {int(idx): char for idx, char in PROCESSING_INFO["idx_to_char"].items()}
char_to_idx = {char: int(idx) for idx, char in PROCESSING_INFO["idx_to_char"].items()}

## Loading Tensorflow Datasets

In [10]:
KEEP_CACHE = True

In [11]:
TRAINING_TYPE = 'Train'
TEST_TYPE = 'Test'
EVAL_TYPE = 'Eval'

In [12]:
dataset_file_paths = paths.assign_wildcard_paths_to_splits_grouped_by_path_position_value(
    wildcard_path = os.path.join(
        DATASET_DUMP_PATH, 
        '*',  # filename
        '*',  # species
        '*'   # istrain
    ),
    path_position = -2,
    splits = {
            TRAINING_TYPE: 0.4,
            TEST_TYPE: 0.5,
            EVAL_TYPE: 0.6
        },
    paths_dump_file = os.path.join(
            DATASET_DUMP_PATH,
            "dataset_file_paths.json"
        ),
    skip_existing = KEEP_CACHE,
    logger = logger
)

print()
print("assigned dataset files:")
visualization.print_list_length_in_dict(dataset_file_paths)

INFO: found file paths dump '../dumps/PXD010000/training_columns/tf_datasets/dataset_file_paths.json'

assigned dataset files:
#Train = 89
e.g.: ../dumps/PXD010000/training_columns/tf_datasets/Biodiversity_C_indologenes_LIB_aerobic_02_03May16_Samwise_16-03-32_mzmlid.parquet/Chryseobacterium_indologenes/Train
#Test = 17
e.g.: ../dumps/PXD010000/training_columns/tf_datasets/Biodiversity_A_cryptum_FeTSB_anaerobic_1_01Jun16_Pippin_16-03-39_mzmlid.parquet/Acidiphilium_cryptum_JF-5/Train
#Eval = 29
e.g.: ../dumps/PXD010000/training_columns/tf_datasets/Biodiversity_B_fragilis_CMcarb_anaerobic_01_01Feb16_Arwen_15-07-13_mzmlid.parquet/Bacteroides_fragilis_638R/Train


### Loading corresponding TF datasets

In [13]:
element_spec = ((tf.TensorSpec(shape=(PROCESSING_INFO['padding_lengths'][MZ],), dtype=tf.float32), 
  tf.TensorSpec(shape=(PROCESSING_INFO['padding_lengths'][INT],), dtype=tf.float32)),
(tf.TensorSpec(shape=(PROCESSING_INFO['padding_lengths'][SEQ],), dtype=tf.int8)))
element_spec

((TensorSpec(shape=(2354,), dtype=tf.float32, name=None),
  TensorSpec(shape=(2354,), dtype=tf.float32, name=None)),
 TensorSpec(shape=(50,), dtype=tf.int8, name=None))

In [14]:
BATCH_SIZE=32

In [15]:
datasets = DatasetLoader(
    element_spec=element_spec,
    batch_size=BATCH_SIZE,
    shuffle_buffer_size=200_000,
    keep_cache=KEEP_CACHE,
    logger=logger
).load_datasets_by_type(dataset_file_paths)
datasets

/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:1640: UserWarning: The `deterministic` argument has no effect unless the `num_parallel_calls` argument is specified.
  warnings.warn("The `deterministic` argument has no effect unless the "


{'Train': <BatchDataset shapes: (((32, 2354), (32, 2354)), (32, 50)), types: ((tf.float32, tf.float32), tf.int8)>,
 'Test': <BatchDataset shapes: (((32, 2354), (32, 2354)), (32, 50)), types: ((tf.float32, tf.float32), tf.int8)>,
 'Eval': <BatchDataset shapes: (((32, 2354), (32, 2354)), (32, 50)), types: ((tf.float32, tf.float32), tf.int8)>}

## Building the Tensorflow Model

In [16]:
input_layers_list, masked_input_layers_list = layers.create_masked_input_layers(
    [
        layers.InputLayerConfiguration(
            name=col,
            shape=PROCESSING_INFO['padding_lengths'][col],
            mask_value=PROCESSING_INFO['padding_characters'][col]
        )
        for col in TRAINING_DATA_COLUMNS
    ]
)
print(input_layers_list)
print(masked_input_layers_list)

[<KerasTensor: shape=(None, 2354) dtype=float32 (created by layer 'mz_array')>, <KerasTensor: shape=(None, 2354) dtype=float32 (created by layer 'intensity_array')>]
[<KerasTensor: shape=(None, 2354) dtype=float32 (created by layer 'masked_mz_array')>, <KerasTensor: shape=(None, 2354) dtype=float32 (created by layer 'masked_intensity_array')>]


In [17]:
masked_loss = losses.MaskedLoss(
    loss_function=tf.keras.losses.sparse_categorical_crossentropy,
    masking_value=tf.constant(
        value=char_to_idx[PROCESSING_INFO['padding_characters'][SEQ]],
        dtype=tf.int8
    )
)

In [18]:
x = masked_input_layers_list[0]
for input_layer in masked_input_layers_list[1:]:
    x = x + input_layer

x = tf.keras.layers.Flatten(name="flattened_masked_inputs")(x)

for _ in range(4):
    x = tf.keras.layers.Dense(2**11)(x)
    x = tf.keras.layers.Dropout(0.1)(x)

x = tf.keras.layers.Dense(PROCESSING_INFO['padding_lengths'][SEQ]*len(idx_to_char))(x)

x = tf.reshape(x,(-1, PROCESSING_INFO['padding_lengths'][SEQ], len(idx_to_char)))

x = tf.keras.activations.softmax(x)

model = tf.keras.Model(inputs=input_layers_list, outputs=x, name='mmproteo')
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=masked_loss,
              metrics=[
                  tf.keras.metrics.SparseCategoricalAccuracy(),
                  tf.keras.metrics.SparseCategoricalCrossentropy()
              ]
             )
model.summary()

Model: "mmproteo"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mz_array (InputLayer)           [(None, 2354)]       0                                            
__________________________________________________________________________________________________
intensity_array (InputLayer)    [(None, 2354)]       0                                            
__________________________________________________________________________________________________
masked_mz_array (Masking)       (None, 2354)         0           mz_array[0][0]                   
__________________________________________________________________________________________________
masked_intensity_array (Masking (None, 2354)         0           intensity_array[0][0]            
___________________________________________________________________________________________

## Training the Tensorflow Model

In [19]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [20]:
TENSORBOARD_LOG_DIR = os.path.join(DUMP_PATH, "tensorboard", "logs")

In [21]:
%tensorboard --logdir $TENSORBOARD_LOG_DIR --bind_all

In [22]:
model.fit(x=datasets[TRAINING_TYPE].repeat(),
          validation_data=datasets[TEST_TYPE].repeat(), 
          validation_steps=500,
          epochs=1,
          steps_per_epoch=1_000,
          callbacks=[
              callbacks.create_tensorboard_callback(
                  tensorboard_log_dir = TENSORBOARD_LOG_DIR,
                  keep_logs = KEEP_CACHE
              )
          ]
         )

1000/1000 [==============================] - 13s 10ms/step - loss: 15.3060 - sparse_categorical_accuracy: 0.0362 - sparse_categorical_crossentropy: 3088450.2500 - val_loss: 15.2695 - val_sparse_categorical_accuracy: 0.0420 - val_sparse_categorical_crossentropy: 6251006.0000


## Evaluating the Tensorflow Model

In [23]:
decode_idx: Callable[[np.ndarray], np.ndarray] = np.vectorize(idx_to_char.get)

In [24]:
evaluator = evaluation.SequenceEvaluator(
    dataset=datasets[EVAL_TYPE],
    decode_func=decode_idx,
    batch_size=BATCH_SIZE
)

In [25]:
evaluator.evaluate_model(model)

1250/1250 [==============================] - 5s 2ms/step - loss: 15.2791 - sparse_categorical_accuracy: 0.0430 - sparse_categorical_crossentropy: 6130912.0000


[15.279091835021973, 0.042952001094818115, 6130912.0]

In [26]:
eval_df, (x_eval, y_eval, y_pred) = evaluator.evaluate_model_visually(
    model=model,
    sample_size=20,
    keep_separator=False,
)
eval_df

,predicted,true
0,R_LSERIQIGG,SGTVTM(Oxidation)DVAK
1,R_LSEMIQIGGRIE,TGDPAEAFEAAQK
2,R_LSEMIDIGGRM,YGIDGGPNPLGR
3,R_LSEMIDIGGRIE,GSFIELDKLVTHR
4,R_LSEMIQIGGRMEFC_K,TDSEVFGFAQINSEHCR
5,R_LSEMIDIGG,M(Oxidation)KYTQEFLGK
6,R_LSEMIQIGGRIEFC_KFNI,TVHELSNEQDMQIQTLSLLR
7,F_LIERIGTGGRMEFC_,FMDEIKKGTDANEALK
8,R_LSERID,FKLPEEK
9,R_LSEMIDIGGRIEFC_KFNIM(Oxidation)T,YIGSDENWEKAEQAIIEACEEK


In [27]:
eval_df.predicted.map(print)
None

R_LSERIQIGG
R_LSEMIQIGGRIE
R_LSEMIDIGGRM
R_LSEMIDIGGRIE
R_LSEMIQIGGRMEFC_K
R_LSEMIDIGG
R_LSEMIQIGGRIEFC_KFNI
F_LIERIGTGGRMEFC_
R_LSERID
R_LSEMIDIGGRIEFC_KFNIM(Oxidation)T
R_LSEMIDIGGRIE
R_LSERIQIGGRMEFC_KFLIM(Oxidation)TNLY
R_LSEMIQIGGRIEFC_KFNIM(Oxidation)TN
R_LSEMIDIGG
F_LIEMIGTGGRMEFP_WVLIM(Oxidation)TD
R_LSEMIDIGG
R_LSERIQIGGRMEFC_WFL
R_LSEMIQIGGRIEF
R_LSERIQM(Oxidation)GGRMEFC_
R_LSEMIDIGGRIE
